# Criação do grid

In [ ]:
import random
import numpy as np

In [ ]:
def generate_random_coordinates(positions):
  coordinates = []
  for _ in range(positions):
    x = random.randint(0, 7)
    y = random.randint(0, 7)
    coordinates.append([x,y])
  return coordinates

In [ ]:
valueGrid =  [[0 for _ in range(8)]for _ in range(8)]

mountains = [[1,3],[2,5],[2,1],[3,7],[4,0],[5,1],[5,4],[6,7],[7,1],[7,5]]

sands = [[0, 1], [6,1], [5, 7], [5, 0], [5, 4], [2, 3], [7, 3], [5, 6], [3, 7], [3, 3], [0, 3], [6, 0], [1, 6], [0, 7]]

# Populando o grid
## Com o valor esperado de cada estado segundo a equação de Bellman

In [ ]:
def possible_moves(x, y):
  moves = []
  if(x == 0):
    if(y == 0):
      moves = ["Down", "Right"]
    elif(y == 7):
      moves = ["Up", "Right"]
    else: moves = ["Up", "Down", "Right"]
  elif(x == 7):
    if(y == 0):
      moves = ["Down", "Left"]
    elif(y == 7):
      moves = ["Up", "Left"]
    else: moves = ["Up", "Down", "Left"]
  elif(y == 0):
    moves = ["Down", "Right", "Left"]
  elif(y == 7):
    moves = ["Up", "Right", "Left"]
  else: moves = ["Up", "Down", "Right", "Left"]
  return moves

def move(direction, x, y):
  if(direction == "Right"):
    x += 1
  elif(direction == "Left"):
    x -= 1
  elif(direction == "Up"):
    y -= 1
  elif(direction == "Down"):
    y += 1
  return x,y

In [ ]:
def bellman_update(x, y, valueGrid, gamma=0.9, goal_reward=100, sand_reward=-10, step_reward=-1):
    if x == 7 and y == 7:
        return goal_reward
    elif is_mountain(x, y, mountains):
        return -np.inf
    elif is_sand(x, y, sands):
        return sand_reward
    else:
        new_value = 0
        for action in possible_moves(x, y):
            new_x, new_y = move(action, x, y)
            if not is_mountain(new_x, new_y, mountains):
              if(is_sand(new_x, new_y, sands)):
                new_value += (1 / len(possible_moves(x, y))) * ((sand_reward + step_reward) + gamma * valueGrid[new_y][new_x])
              else: new_value += (1 / len(possible_moves(x, y))) * ((step_reward) + gamma * valueGrid[new_y][new_x])
        return round(new_value, 2)

def value_iteration( valueGrid, epsilon=1e-6):
  while True:
    delta = 0
    for y in range(8):
      for x in range(8):
        old_value = valueGrid[y][x]
        new_value = bellman_update(x, y, valueGrid)
        valueGrid[y][x] = new_value
        delta = max(delta, abs(new_value - old_value))
    if delta < epsilon:
      break

In [ ]:
def is_mountain(x,y, mountains):
  return [x,y] in mountains

def is_sand(x, y, sands):
    return (x, y) in sands

In [ ]:
value_iteration(valueGrid)

#  Função para visualização do grid

In [ ]:
valueGrid #antes da atualização de valores

[[-5.28, -4.48, -2.69, -2.28, -inf, -1.03, -2.33, -1.55],
 [-5.03, -3.63, -inf, -2.7, -2.05, -inf, -1.86, -inf],
 [-4.52, -3.3, -3.2, -4.33, -4.21, -3.3, -3.72, -2.93],
 [-3.41, -inf, -3.25, -4.71, -4.58, -3.42, -3.98, -3.82],
 [-4.64, -3.39, -3.21, -4.31, -3.58, -inf, -2.29, -2.5],
 [-5.35, -3.88, -inf, -3.22, -3.7, -1.94, -0.38, -inf],
 [-5.99, -5.17, -3.31, -2.95, -3.25, -1.22, 5.89, 31.1],
 [-6.11, -5.37, -3.27, -inf, -2.15, -1.68, -inf, 100]]

In [ ]:
valueGrid #após a atualização de valores

[[-5.28, -4.48, -2.69, -2.28, -inf, -1.03, -2.33, -1.55],
 [-5.03, -3.63, -inf, -2.7, -2.05, -inf, -1.86, -inf],
 [-4.52, -3.3, -3.2, -4.33, -4.21, -3.3, -3.72, -2.93],
 [-3.41, -inf, -3.25, -4.71, -4.58, -3.42, -3.98, -3.82],
 [-4.64, -3.39, -3.21, -4.31, -3.58, -inf, -2.29, -2.5],
 [-5.35, -3.88, -inf, -3.22, -3.7, -1.94, -0.38, -inf],
 [-5.99, -5.17, -3.31, -2.95, -3.25, -1.22, 5.89, 31.1],
 [-6.11, -5.37, -3.27, -inf, -2.15, -1.68, -inf, 100]]

# Função com o conjunto de ações que o agente pode realizar

In [ ]:
class Agente:
    def __init__(self):
        self.x = 0
        self.y = 0

    def find_greedy_route(self, grid):
        route = []
        while (self.x, self.y) != (7, 7):
            best_value = -float('inf')
            best_direction = None
            for action in possible_moves(self.x, self.y):
                new_x, new_y = move(action, self.x, self.y)
                if grid[new_y][new_x] > best_value and (not route or action != opposite(route[-1])):
                    best_value = grid[new_y][new_x]
                    best_direction = action
            route.append(best_direction)
            self.x, self.y = move(best_direction, self.x, self.y)
        return route

def opposite(action):
    if action == "Up":
        return "Down"
    elif action == "Down":
        return "Up"
    elif action == "Left":
        return "Right"
    elif action == "Right":
        return "Left"
    else:
        return None

# Comparação de rotas


In [ ]:
def calculate_route_sum(grid, route):
  y, x = 0, 0  # Começando do canto superior esquerdo do grid
  total_sum = grid[0][0]

  for move in route:
    if move == "Right":
      x += 1
    elif move == "Down":
      y += 1
    total_sum += grid[y][x]
  return total_sum

In [ ]:
agente = Agente()
route1 = agente.find_greedy_route(valueGrid)
calculate_route_sum(valueGrid, route1)

99.93

In [ ]:
route2 = ["Right", "Down", "Down", "Right", "Right", "Right", "Down", "Down", "Down", "Right", "Right", "Down", "Right", "Down"]

calculate_route_sum(valueGrid, route2)

94.38

Portanto, vemos que a rota1, calculada a partir de uma política gulosa que decidia qual ação tomar com base no maior valor possível de estado em que poderia alcançar no momento, possui uma recompensa maior do que a rota2, definida manualmente a fim de exemplo